In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/red_data_no_webp.zip



In [ ]:
import os

def list_file_extensions(directory):
    # Store unique file extensions
    extensions = set()

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Get the file extension and add to the set (ignoring empty extensions)
            ext = os.path.splitext(file)[1]
            if ext:  # Only add extensions that are not empty
                extensions.add(ext.lower())

    # Return sorted list of unique extensions
    return sorted(extensions)

# Example usage:
directory_path = input("Enter the directory path: ")
extensions = list_file_extensions('/content/red_data/img/')

print("Unique file extensions in the directory:")
for ext in extensions:
    print(ext)


Enter the directory path: fvgbnm,.
Unique file extensions in the directory:
.jpeg
.jpg


In [ ]:
import os

extracted_files = os.listdir('/content/red_data/img/')
print(extracted_files)


['Metal', 'Plastic-Regular', 'Glass', 'Paperboard', 'Plastic-Polystyrene']


In [ ]:
import tensorflow as tf
import numpy as np
import os
import pathlib
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.utils import class_weight

# Set image dimensions
img_height = 240
img_width = 240
batch_size = 8

# Path to your image directory
str_path = '/content/red_data/img/'
data_dir = pathlib.Path(str_path)

# Load the datasets without applying cache/prefetch initially
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)




val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Now access class_names before any dataset transformation
class_names = train_ds.class_names
print(class_names)  # You should be able to access class names here

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Apply caching and prefetching after accessing class names
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


num_classes = 5
for images, labels in train_ds.take(1):
    print("Image shape:", images.shape)
# Define the base pre-trained model (ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.summary()
# Freeze the base model (all layers initially frozen)
base_model.trainable = False

# Create the custom model on top of the pre-trained model
model = models.Sequential([
    base_model,  # Pre-trained ResNet50 model
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Output layer with softmax
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from sklearn.metrics import classification_report

def custom_metric(y_true, y_pred):
    return classification_report(y_true, y_pred, target_names=class_names)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

]

# Train the model with class weights
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    #class_weight=class_weights_dict
)



# After training the new layers, unfreeze some layers of the base model to fine-tune
base_model.trainable = True

# Freeze the earlier layers of ResNet50 to prevent them from being modified too much during fine-tuning
for layer in base_model.layers[:100]:  # You can experiment with which layers to unfreeze
    layer.trainable = False

from sklearn.metrics import classification_report

def custom_metric(y_true, y_pred):
    return classification_report(y_true, y_pred, target_names=class_names)

# Re-compile the model for fine-tuning with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    #class_weight=class_weights_dict
)

# Save the fine-tuned model
model.save('recyclebot.keras')
print("Transfer learning completed and model saved.")
